In [6]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
import os

In [7]:
df_dk = pd.read_csv("ParlaMint-DK-SENT.csv", delimiter = ",", dtype = object)
df_dk = df_dk[df_dk["Speaker_role"]!= "Chairperson"]
df_dk = df_dk.drop(df_dk[(df_dk["Speaker_Minister"] == "-") & (df_dk["Speaker_MP"] == "notMP")].index)
df_dk["Word_total"] = df_dk["Word_total"].astype(float)
df_dk = df_dk[df_dk["Word_total"] > 10]
df_dk["Score"] = df_dk["Score"].astype(float)
df_dk["Sentiment"] = df_dk["Sentiment"].astype(float)
#df_dk["Negscore"] = df_dk["Negscore"].astype(float)/df_dk["Word_total"]
#df_dk = df_dk[df_dk["Comission"] == "False"]
pd.set_option('display.max_colwidth', None)

In [8]:
session = df_dk["Title"].str.split(",", expand = True)[1].str.lstrip()
meeting = df_dk["Title"].str.split(",", expand = True)[2].str.split(" ", expand = True)[1]

In [9]:
df_dk["Filename"] = "ParlaMint-DK_" + df_dk["Date"].str.cat([session, meeting], sep = "-") + ".ana.sent.csv"

In [10]:
filedict = dict(zip(df_dk["ID"], df_dk["Filename"]))

In [52]:
random.seed(6)
sample = random.sample(df_dk["ID"].to_list(), 1000)

In [41]:
root = "ParlaMint-DK.TEI.CSV.SENT"
maximum = 100

with open("gold_standard_DK.txt", "w") as f:
    for speech_id in sample:
        year = speech_id[13:17]
        file = str(filedict[speech_id])
        filename = os.path.join(root, year, file)
        df = pd.read_csv(filename, delimiter = ",", index_col = 0, dtype = object)
        df["segsent"] = df["seg"] + "." + df["sent"]
        speecharray = df[df["id"] == speech_id]
                    
        speech_length = len(speecharray)
        if speech_length <= maximum:
            speech = " ".join(speecharray["token"].to_list())
            score = str(speecharray["sentiment"].astype(np.float64).sum() / len(speecharray))
            f.write("\t".join([speech_id, str(speech_length), score, speech]))
            f.write("\n")
            
        else:
            last = speecharray["segsent"].iloc[maximum]
            last_index = speecharray.loc[speecharray["segsent"] == last].index[-1]
            speech_cut = speecharray.loc[:last_index]
            score = str(speech_cut["sentiment"].astype(np.float64).sum() / len(speech_cut))
            speech = " ".join(speech_cut["token"].to_list())
            f.write("\t".join([speech_id, str(speech_length), score, speech]))
            f.write("\n")

### Gold Standard CSV

In [23]:
gs_root = "/Users/yananikolova/Dropbox/ITCOG/Thesis/"
gs_score = pd.read_csv(gs_root + "annotated_DK_gold.tsv", delimiter = "\t", encoding = "utf-16le", usecols = [0,1,2,3,4,5], 
                names = ["ID", "Sentiment", "Speech", "Annotation", "Note", "2nd Annot"], dtype = object).dropna(subset = "Annotation")[:300]

In [24]:
sample = gs_score["ID"].to_list()

In [25]:
root = "ParlaMint-DK.TEI.CSV.SENT"
maximum = 100

with open("gold_standard_DK.csv", "w") as file:
    file.write(",id,token,lemma,NE,seg,sent,word,pos,msd1,msd2,msd3,msd4,msd5,msd6,msd7,sentiment,segsent\n")


for speech_id in sample:
    year = speech_id[13:17]
    file = str(filedict[speech_id])
    filename = os.path.join(root, year, file)
    df = pd.read_csv(filename, delimiter = ",", index_col = 0, dtype = object)
    df["segsent"] = df["seg"] + "." + df["sent"]
    speecharray = df[df["id"] == speech_id]

    speech_length = len(speecharray)
    if speech_length <= maximum:
        speecharray.to_csv("gold_standard_DK.csv", mode='a', header=False)
        
    else:
        last = speecharray["segsent"].iloc[maximum]
        last_index = speecharray.loc[speecharray["segsent"] == last].index[-1]
        speech_cut = speecharray.loc[:last_index]
        speech_cut.to_csv("gold_standard_DK.csv", mode='a', header=False)

In [26]:
filedict["ParlaMint-DK_20181211164251"]

'ParlaMint-DK_2018-12-11-20181-M34.ana.sent.csv'

In [27]:
pd.read_csv(filename, delimiter = ",", index_col = 0, dtype = object)

,id,token,lemma,NE,seg,sent,word,pos,msd1,msd2,msd3,msd4,msd5,msd6,msd7,sentiment
0,ParlaMint-DK_20180515130030,Mødet,møde,None,seg1,1,1,NOUN,Definite=Def,Gender=Neut,Number=Sing,NaN,NaN,NaN,NaN,NaN
1,ParlaMint-DK_20180515130030,er,være,None,seg1,1,2,AUX,Mood=Ind,Tense=Pres,VerbForm=Fin,Voice=Act,NaN,NaN,NaN,NaN
2,ParlaMint-DK_20180515130030,åbnet,åbne,None,seg1,1,3,VERB,Definite=Ind,Number=Sing,Tense=Past,VerbForm=Part,NaN,NaN,NaN,NaN
3,ParlaMint-DK_20180515130030,.,.,None,seg1,1,4,PUNCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ParlaMint-DK_20180515130030,Medlemmer,medlem,None,seg1,2,1,NOUN,Definite=Ind,Gender=Neut,Number=Plur,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59831,ParlaMint-DK_20180515183207,fremgår,fremgå,None,seg366,2,23,VERB,Mood=Ind,Tense=Pres,VerbForm=Fin,Voice=Act,NaN,NaN,NaN,NaN
59832,ParlaMint-DK_20180515183207,af,af,None,seg366,2,24,ADP,AdpType=Prep,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59833,ParlaMint-DK_20180515183207,Folketingets,folketing,None,seg366,2,25,NOUN,Case=Gen,Definite=Def,Gender=Neut,Number=Sing,NaN,NaN,NaN,NaN
59834,ParlaMint-DK_20180515183207,hjemmeside,hjemmeside,None,seg366,2,26,NOUN,Definite=Ind,Gender=Com,Number=Sing,NaN,NaN,NaN,NaN,NaN
